<a href="https://colab.research.google.com/github/Mu-niu13/Emotion-Analysis/blob/main/lib/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up Connection to Github

In [1]:
## install git and configure(so we know who commit the code)
# !apt-get install git
# !git config --global user.name "USER_NAME"
# !git config --global user.email "EMAIL"

# access google drive
# from google.colab import drive
# drive.mount('/content/drive')

# clone repo
!git clone https://github.com/Mu-niu13/Emotion-Analysis.git
%cd Emotion-Analysis

# set up personal access for push/pull
from getpass import getpass
token = getpass('Enter your GitHub PAT:')
!git remote set-url origin https://{token}@github.com/Mu-niu13/Emotion-Analysis.git

Cloning into 'Emotion-Analysis'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 121 (delta 76), reused 61 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (121/121), 11.95 MiB | 10.44 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/Emotion-Analysis
Enter your GitHub PAT:··········


In [ ]:
!make install

Installing dependencies...
# Install python3-venv if not present (specific to Debian/Ubuntu systems like Colab)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,616 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,223 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelea

## Model Training

#### 1. Set Up

In [19]:
# import libraries
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

# import custom modules
from lib.data_preprocessing import *
from lib.model import EmotionClassifier
from lib.train import train_epoch
from lib.evaluate import eval_model

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


#### 2. Training Loop

In [6]:
# load data
df = load_data('/content/Emotion-Analysis/data/merged_filtered_data.csv')

# get label columns
label_columns = get_label_columns(df)
n_classes = len(label_columns)

# train val split
df_train, df_val = train_test_split(df, test_size=0.1, random_state=42)

# init tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# create data loaders
MAX_LEN = 150
BATCH_SIZE = 64

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE, label_columns, True)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE, label_columns, False)

# init model
model = EmotionClassifier(n_classes=n_classes, model_name='bert-base-uncased')
model = model.to(device)

# define optimizer with parameter groups
optimizer = AdamW([
    {'params': model.model.parameters(), 'lr': 2e-5},
    {'params': model.layer_norm.parameters(), 'lr': 1e-4},
    {'params': model.fc1.parameters(), 'lr': 1e-4},
    {'params': model.fc2.parameters(), 'lr': 1e-4},
    {'params': model.fc3.parameters(), 'lr': 1e-4},
], weight_decay=1e-2, correct_bias=False)

# define total steps and warmup steps
EPOCHS = 5
total_steps = len(train_data_loader) * EPOCHS
warmup_steps = int(0.1 * total_steps)

# set scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# define loss
loss_fn = nn.CrossEntropyLoss().to(device)

# enable Automatic Mixed Precision
scaler = torch.cuda.amp.GradScaler()

# set up early stopping
best_val_loss = float('inf')
epochs_no_improve = 0
n_epochs_stop = 3
best_model_state = None

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')

    # training
    train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        scaler
    )
    print(f'Train loss {train_loss}')

    # validation
    val_loss, outputs, targets = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device
    )
    print(f'Validation loss {val_loss}')

    # compute metrics
    outputs = torch.softmax(outputs, dim=1)
    outputs_np = outputs.cpu().numpy()
    targets_np = targets.cpu().numpy()

    max_prob_classes = outputs_np.argmax(axis=1)
    true_classes = targets_np

    overall_accuracy = (max_prob_classes == true_classes).mean()
    print(f'Overall Accuracy: {overall_accuracy:.4f}')

    report = classification_report(
        true_classes,
        max_prob_classes,
        target_names=label_columns,
        zero_division=0
    )
    print(report)

    # early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        best_model_state = model.state_dict()
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= n_epochs_stop:
            print('Early stopping!')
            model.load_state_dict(best_model_state)
            torch.save(model.state_dict(), 'best_model_state.bin')
            break

# save best model after training
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    torch.save(model.state_dict(), 'best_model_state.bin')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: T

Epoch 1/5


...

Validation loss 1.4720158569629376
Overall Accuracy: 0.4757
                      precision    recall  f1-score   support

             neutral       0.53      0.49      0.51      5478
positive recognition       0.50      0.53      0.51      3399
               anger       0.42      0.47      0.44      2018
             sadness       0.41      0.40      0.41       690
             anxiety       0.41      0.46      0.44       361
              regret       0.33      0.28      0.30       773
           happiness       0.59      0.60      0.60      3423
          discomfort       0.36      0.34      0.35      1969
           affection       0.45      0.51      0.47       950
           curiosity       0.35      0.42      0.38       890
            surprise       0.34      0.24      0.29       831

            accuracy                           0.48     20782
           macro avg       0.43      0.43      0.43     20782
        weighted avg       0.48      0.48      0.47     20782

Epoch 5

  0%|          | 0/2923 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/content/Emotion-Analysis/lib/train.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 2923/2923 [18:44<00:00,  2.60it/s]


Train loss 1.3165330503886585


100%|██████████| 325/325 [02:42<00:00,  2.00it/s]


Validation loss 1.485069083067087
Overall Accuracy: 0.4778
                      precision    recall  f1-score   support

             neutral       0.52      0.51      0.51      5478
positive recognition       0.52      0.50      0.51      3399
               anger       0.41      0.48      0.44      2018
             sadness       0.43      0.39      0.41       690
             anxiety       0.41      0.49      0.45       361
              regret       0.33      0.29      0.31       773
           happiness       0.60      0.60      0.60      3423
          discomfort       0.36      0.35      0.36      1969
           affection       0.44      0.53      0.48       950
           curiosity       0.37      0.41      0.39       890
            surprise       0.35      0.24      0.29       831

            accuracy                           0.48     20782
           macro avg       0.43      0.44      0.43     20782
        weighted avg       0.48      0.48      0.48     20782

Early st

#### 3. Performance on "Read" Data

In [20]:
def evaluate_on_gpt_data(model, data_loader, device):
    model.eval()
    all_outputs = []
    all_labels = []
    texts = []

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            all_outputs.append(outputs)
            all_labels.append(labels)

    all_outputs = torch.cat(all_outputs)
    all_labels = torch.cat(all_labels)

    return all_outputs, all_labels

In [26]:
# load model
model = EmotionClassifier(n_classes=11, model_name='bert-base-uncased')
model = model.to(device)
model.load_state_dict(torch.load('best_model_state.bin'))
model.eval()

# initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# define columns mapping
label_columns = [col for col in gpt_df.columns if col not in ['id', 'text', 'example_very_unclear']]
label_mapping = {index: label for index, label in enumerate(label_columns)}

# load GPT data
gpt_df = load_data('/content/Emotion-Analysis/data/GPT_generated_emotion_data.csv')
gpt_data_loader = create_data_loader(gpt_df, tokenizer, MAX_LEN, BATCH_SIZE, label_columns, shuff = False)

# evaluate on GPT data
outputs, labels = evaluate_on_gpt_data(model, gpt_data_loader, device)

# get predictions
outputs = torch.softmax(outputs, dim=1)
outputs_np = outputs.cpu().numpy()
labels_np = labels.cpu().numpy()

# predicted classes
predicted_classes = outputs_np.argmax(axis=1)
true_classes = labels_np

# overall accuracy
accuracy = (predicted_classes == true_classes).mean()
print(f'Accuracy on GPT-generated data: {accuracy:.4f}')

# misclassified examples
misclassified_indices = np.where(predicted_classes != true_classes)[0]
misclassified_texts = [gpt_df['text'].to_list()[i] for i in misclassified_indices]
misclassified_true_labels = [label_mapping[int(true_classes[i])] for i in misclassified_indices]
misclassified_predicted_labels = [label_mapping[int(predicted_classes[i])] for i in misclassified_indices]

misclassified_df = pd.DataFrame({
    'text': misclassified_texts,
    'true_label': misclassified_true_labels,
    'predicted_label': misclassified_predicted_labels
})

print(f'Number of misclassified examples: {len(misclassified_df)}')
print("Misclassified examples:")
misclassified_df

<ipython-input-26-4386759811e3>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_state.bin'))
/usr/local/lib/python3.10/dist-pac

Accuracy on GPT-generated data: 0.4906
Number of misclassified examples: 27
Misclassified examples:


,text,true_label,predicted_label
0,This is the best day I've had in ages.,happiness,positive recognition
1,This revelation is truly mind-blowing!,surprise,positive recognition
2,Why can't things just go smoothly for once?,anger,curiosity
3,I feel like a dark cloud is over me.,sadness,anxiety
4,I feel so out of place right now.,discomfort,regret
5,Your presence brings me so much joy.,affection,happiness
6,My heart races every time I consider it.,anxiety,neutral
7,Nothing surprising in this statement.,neutral,surprise
8,The thought of tomorrow keeps me up at night.,anxiety,neutral
9,I can't stop wondering about the next step.,curiosity,surprise


No charts were generated by quickchart


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_2['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)